In [1]:
import pandas as pd
import random
import numpy as np
import pyodbc
from datetime import date, datetime
import openpyxl
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException
import re 

list_numbers = []
list_date_start =[]
list_info = []
list_judge=[]
list_date_des =[]
list_des=[]
list_post=[]
list_date_vst=[]
list_one=[]
list_two=[]
list_three=[]
list_post_2 = []

data = pd.read_excel(r"C:\Users\BogatyrevVA\Desktop\Парсинг сайтов\TZ (входной файл).xlsx")
# data = pd.read_excel(r"/Users/allastepannikova/Downloads/TZ (входной файл).xlsx")
vhod = data['Номер'].to_list()
url_list = data['Администратор'].to_list()
money = data['Сумма'].to_list()

vhod = vhod[98:109]
url_list = url_list[98:109]
money = money[98:109]
browser = webdriver.Chrome()
for ind,inputt in enumerate(vhod):
    infor = 0
    url = 'http://' + str(url_list[ind]) + '.mo.msudrf.ru/modules.php?name=sud_delo&adm_case__CASE_NUMBERSS='
    inputting = inputt.split('/')
    url = f'http://{str(url_list[ind])}.mo.msudrf.ru/modules.php?name=sud_delo&adm_case__CASE_NUMBERSS={inputting[0]}%2F{inputting[1]}&delo_id=1500001&op=sf'
    browser.get(url)
    time.sleep(random.randint(0,3))
    try:
        search = browser.find_element(By.XPATH, '/html/body/div[1]/div[3]/div[3]/div[7]')
    except:
        time.sleep(20)
    finally:
        soup = BeautifulSoup(browser.page_source,'html.parser')
        list_of = soup.find_all('td', class_='vert-align')
        list_p=soup.find_all('td', class_="lawcase-number-td")
        list_numbers.append(list_p[1].text)
        list_date_start.append(list_of[0].text)
        list_info.append(list_of[1].text)
        list_judge.append(list_of[2].text)
        list_date_des.append(list_of[3].text)
        list_des.append(list_of[4].text)


    try:
        m=list_of[6].find_all('a', class_="document-link")
        if m:
            url_act = url + m[0].attrs['href']
            browser.get(url_act)
            time.sleep(random.randint(3,4))
            soup_act = BeautifulSoup(browser.page_source,'html.parser')
            postanovil=soup_act.find_all('p', class_="MsoNormal")
            for i in range(len(postanovil)):
                postanovil[i] = postanovil[i].text
                if ('признать виновной' in postanovil[i]) or ('признать виновным' in postanovil[i]):
                    infor = 1
                    list_post.append(postanovil[i])
                    list_post_2.append(postanovil[i+1].text)

        list_date_vst.append(list_of[5].text)
    except:
        try:
            m=list_of[5].find_all('a', class_="document-link")
            if m:
                url_act = url + m[0].attrs['href']
                browser.get(url_act)
                time.sleep(random.randint(3,4))
                soup_act = BeautifulSoup(browser.page_source,'html.parser')
                postanovil=soup_act.find_all('p', class_="MsoNormal")
                for i in range(len(postanovil)):
                    postanovil[i] = postanovil[i].text
                    if ('признать виновной' in postanovil[i]) or ('признать виновным' in postanovil[i]):
                        infor=1
                        list_post.append(postanovil[i])
                        list_post_2.append(postanovil[i+1].text)

            list_date_vst.append('нет инфо')
        except:
            list_date_vst.append('нет инфо')


    if infor == 0:
        list_post.append('нет инфо')
        list_post_2.append('нет инфо')
    list_one.append(vhod[ind])
    list_two.append(url_list[ind])
    list_three.append(money[ind])
    time.sleep(random.randint(0,3))




browser.quit()

matrix = np.array([list_numbers, list_date_start, list_info, list_judge, list_date_des, list_des, list_date_vst, list_post,list_two, list_three, list_post_2])
matrix = matrix.transpose()
df = pd.DataFrame(data=(matrix), columns=['Номер дела','Дата поступления','Информация по делу','Судья','Дата решения','Решение','Дата вступления в законную силу','Судебные акты','Номер района', 'Сумма', 'Получатель'])

# здесь путь к результирующему файлу
file_= r"C:\Users\BogatyrevVA\Desktop\Парсинг сайтов\судебные делишки.xlsx"
# file_= r"/Users/allastepannikova/Downloads/shturman.xlsx"

# wb = openpyxl.Workbook()
# wb.save(file_)
with pd.ExcelWriter(file_, mode='a', engine="openpyxl",if_sheet_exists='overlay') as writer:  
    df.to_excel(writer, sheet_name='Sheet', startrow=99, index=False, header=False)